In [32]:
#!jupyter nbconvert --to script userdatapreprocessing.ipynb

In [1]:
import pandas as pd
import os 
import inspect

# Put all datasets in 'path-to-repo/data' directory please
path = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) # script directory
path += '/data/'


os.chdir(path)

with open('yelp_academic_dataset_checkin.json', 'rb') as f:
    data = f.readlines()

data = map(lambda x: x.rstrip(), data)

data_json_str = "[" + ','.join(data) + "]"

data_df = pd.read_json(data_json_str)

In [2]:
path

'/afs/inf.ed.ac.uk/user/s13/s1338673/Desktop/STN/data/'

In [3]:
newcols = []
for i in range(0,24):
    for j in range (0,6):
        newcols.append(str(i) + '-' + str(j)) 

In [4]:
for newcol in newcols:
    data_df[newcol] = 0

In [5]:
def fintot(row):
    return sum(row['checkin_info'].values())

data_df['total'] = data_df.apply(lambda row: fintot(row),axis=1)

In [6]:
def fincols(row):
    for newcol in newcols:
        if (newcol in row['checkin_info'].keys()):
            row[newcol] = row['checkin_info'][newcol]
    return row
 
              
ndata_df = data_df.apply(lambda row:fincols(row) ,axis=1)

In [7]:
ndata_df['business_id'] = data_df['business_id']

In [8]:
morningcol = []
nooncol= []
nightcol = []
latenightcol = []
weekendcol = []
weekcol = []
for newcol in newcols:
    if (int(newcol[:-2]) in range(6,12)):
        morningcol.append(newcol)
    if (int(newcol[:-2]) in range(12,17)):
            nooncol.append(newcol)
    if (int(newcol[:-2]) in range(17,23)):
            nightcol.append(newcol)
    if (int(newcol[:-2]) in range(21,24)) or (int(newcol[:-2]) in range(0,4)):
            latenightcol.append(newcol)
    if (int(newcol[-1]) in range(0,5)): 
            weekcol.append(newcol)
    if (int(newcol[-1]) in range(5,7)): 
            weekendcol.append(newcol)

In [9]:
def addagg(row, whichcols):
    s = 0
    for col in whichcols:
        s += row[col]
    return s

In [10]:
allcols = list(ndata_df.columns.values)[4:]

In [11]:
ndata_df['morningci'] = ndata_df.apply(lambda row:addagg(row , morningcol) ,axis=1)
ndata_df['noonci'] = ndata_df.apply(lambda row:addagg(row , nooncol) ,axis=1)
ndata_df['nightci'] = ndata_df.apply(lambda row:addagg(row , nightcol) ,axis=1)
ndata_df['latenightci'] = ndata_df.apply(lambda row:addagg(row , latenightcol) ,axis=1)
ndata_df['weekci'] = ndata_df.apply(lambda row:addagg(row , weekcol) ,axis=1)
ndata_df['weekendci'] = ndata_df.apply(lambda row:addagg(row , weekendcol) ,axis=1)

In [12]:
ndata_df.head()

,business_id,checkin_info,type,0-0,0-1,0-2,0-3,0-4,0-5,1-0,...,23-3,23-4,23-5,total,morningci,noonci,nightci,latenightci,weekci,weekendci
0,cE27W9VPgO88Qxe4ol6y_g,"{u'9-5': 1, u'7-5': 1, u'13-3': 1, u'17-6': 1,...",checkin,0,0,0,0,0,0,0,...,0,0,0,9,3,2,2,0,5,2
1,mVHrayjG3uZ_RLHkLj-AMg,"{u'11-5': 1, u'9-5': 2, u'9-4': 3, u'15-1': 1,...",checkin,0,0,0,0,0,0,0,...,0,0,0,28,13,12,3,2,21,7
2,mYSpR_SLPgUVymYOvTQd_Q,"{u'5-5': 1, u'5-4': 1, u'9-3': 1, u'11-4': 1, ...",checkin,0,0,0,0,0,0,0,...,0,0,0,8,3,2,1,0,6,2
3,KayYbHCt-RkbGcPdGOThNg,"{u'23-1': 1, u'11-2': 2, u'21-4': 1, u'16-1': ...",checkin,0,0,0,0,0,0,0,...,0,0,0,65,20,27,16,6,56,8
4,b12U9TFESStdy7CsTtcOeg,"{u'5-5': 2, u'13-2': 1, u'8-6': 1, u'5-1': 2, ...",checkin,0,0,0,0,0,0,0,...,0,0,0,7,1,1,0,0,4,2


In [13]:
def removehip(row):
    if(row['business_id'][:2]=='--'):
        return row['business_id'][2:]
    if(row['business_id'][0]=='-'):
        return row['business_id'][1:]

In [14]:
#ndata_df['business_id'] = ndata_df.apply(lambda row:removehip(row) ,axis=1)

In [15]:
#ndata_df.groupby(by=['business_id'])[allcols].sum()

In [16]:
aggdata_df = ndata_df.groupby(by=['business_id'])[allcols].sum()

In [17]:
aggdata_df.head()

,0-1,0-2,0-3,0-4,0-5,1-0,1-1,1-2,1-3,1-4,...,22-3,22-4,22-5,23-0,23-1,23-2,23-3,23-4,23-5,total
business_id,,,,,,,,,,,,,,,,,,,,,
--1emggGHgoG6ipd_RMb-g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
--4Pe8BZ6gj57VFL5mUE8g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,35
--5jkZ3-nUPZxUvtcbr8Uw,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,546
--AKjxBmhm9DWrh-e0hTOw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
--BlvDO_RG2yElKu9XA1_g,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,336


In [31]:
import pickle

file5 = open(r'agguserdata.pkl', 'wb')
pickle.dump(aggdata_df, file5)
file5.close()